In [2]:
from langchain_ollama import ChatOllama
import re

llm = ChatOllama(
    model="llama3.2:latest",
    temperature=0
)
# Made for deepseek model
def remove_think_tags(text):
    """
    Elimina todas las ocurrencias de las etiquetas <think> y su contenido de la cadena proporcionada.
    
    Args:
    text (str): La cadena de texto de la cual se eliminarán las etiquetas <think>.
    
    Returns:
    str: La cadena de texto sin las etiquetas <think> y su contenido.
    """
    cleaned_text = re.sub(r'<think>.*?</think>', '', text, flags=re.DOTALL)
    return cleaned_text.strip()

In [3]:
from typing import Literal, TypedDict
from pydantic import BaseModel, Field

agents = ["main_agent","about_agent","not_related_agent"]
options = agents + ["FINISH"]
system_prompt = (
    " You are part of a team of workers tasked with answering questions, you are a router and you have to decide which worker to send the question to next."
    f" These are the following workers: {agents},"
    " * main_agent: ONLY for questions specifically about routers, telecommunications, networking equipment and particularly the mikrotik brand."
    " * about_agent: STRICTLY LIMITED to basic conversational elements (formal greetings like hello/hi/hola, goodbyes like bye/adios, thanks, laughter like 'haha'/'jajaja') and direct questions about the bot itself (name, purpose, capabilities, architecture, developer team)."
    " * other: for ALL other questions including personal questions, emotional queries, opinions, and ANY topic not directly related to networking equipment or basic bot information."
    " Be very conservative in your classification. When in doubt, classify as 'other'.\n"
    """
    Examples:
    user: que es un router?
    next: main_agent
    think: The question is asking what a router is, which is directly related to networking equipment. This is clearly about routers and telecommunications, so it should be sent to main_agent.
    
    user: deseo que me compares los modelos xyz y zed para una compra corporativa?
    next: main_agent
    think: The question is comparing networking equipment models for a corporate purchase, which directly relates to telecommunications hardware. So it should be sent to main_agent.
    
    user: Hi, what's your name?
    next: about_agent
    think: This is a basic greeting with a direct question about the bot's identity, which falls under bot information. So it should be sent to about_agent.
    
    user: Goodbye!
    next: about_agent
    think: This is a basic farewell message which falls under the limited conversational interactions handled by about_agent.
    
    user: Adios!
    next: about_agent
    think: This is a basic Spanish farewell message which falls under the limited conversational interactions handled by about_agent.
    
    user: jajaja
    next: about_agent
    think: This is basic Spanish laughter/expression similar to "hahaha", which is a simple conversational element handled by about_agent.
    
    user: me quieres?
    next: other
    think: This is a personal/emotional question asking if the bot loves/likes the user. This goes beyond basic greetings or bot information questions, so it should be sent to other.
    
    user: ¿qué opinas de la política?
    next: other
    think: This asks for an opinion on politics, which is not related to networking equipment or basic bot information. It should be sent to other.
    
    user: el iphone 5 es mejor que el 6?
    next: other
    think: The question is about comparing iPhone models, which is not related to networking equipment or basic bot information. It should be sent to other.
    """
)
print(system_prompt)

class Router(TypedDict):
    """Worker to route to next"""
    next: Literal["main_agent", "test_agent","about_agent","not_related_agent"]
    think: str = Field(description="All thoughts to be shared with the next worker")

llm_structured = llm.with_structured_output(Router,method= 'json_schema')

 You are part of a team of workers tasked with answering questions, you are a router and you have to decide which worker to send the question to next. These are the following workers: ['main_agent', 'about_agent', 'not_related_agent'], * main_agent: ONLY for questions specifically about routers, telecommunications, networking equipment and particularly the mikrotik brand. * about_agent: STRICTLY LIMITED to basic conversational elements (formal greetings like hello/hi/hola, goodbyes like bye/adios, thanks, laughter like 'haha'/'jajaja') and direct questions about the bot itself (name, purpose, capabilities, architecture, developer team). * other: for ALL other questions including personal questions, emotional queries, opinions, and ANY topic not directly related to networking equipment or basic bot information. Be very conservative in your classification. When in doubt, classify as 'other'.

    Examples:
    user: que es un router?
    next: main_agent
    think: The question is asking

In [6]:
messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": "user: "},
    ]
llm_structured.invoke(messages)

{'next': 'main_agent',
 'think': 'The question is asking for a comparison between two specific products (iPhone 5 and iPhone 6), which is not directly related to networking equipment or basic bot information.'}